#Reimplementation

In [1]:
!pip install --upgrade xarray mizani plotnine
!pip install --upgrade cudf-cu12 dask-cudf-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.1.2
    Uninstalling xarray-2025.1.2:
      Successfully uninstalled xarray-2025.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1


In [2]:
!pip install shap dice-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.0 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you 

In [3]:
!pip install numpy==1.25.0 pandas==1.5.3 --upgrade --force-reinstall

  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.9 MB/s eta 0:00:00
Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.1
    Uninstalling pytz-2025.1:
      Successfully uninstalled pytz-2025.1
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dat

In [1]:
import numpy as np
import pandas as pd
import urllib.request
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import dice_ml
from dice_ml.utils import helpers  # helper functions

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

import pickle


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
dataset = helpers.load_adult_income_dataset()

In [5]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [6]:
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [7]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=42,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

In [8]:
len(x_train), len(x_test)

(20838, 5210)

In [9]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
continuous_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical),
        ('num', continuous_transformer, numerical)
    ]
)

In [10]:
logreg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=200))
])

logreg.fit(x_train, y_train)

y_pred_linear = logreg.predict(x_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_linear))

Logistic Regression Accuracy: 0.8197696737044146


In [11]:
from sklearn.neural_network import MLPClassifier
nonlinear = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(
        hidden_layer_sizes=(16,),
        activation='relu',
        solver='adam',
        learning_rate_init=0.001,
        max_iter=200,
        random_state=42
    ))
])

nonlinear.fit(x_train, y_train)

y_pred_nonlinear = nonlinear.predict(x_test)
print("Non-linear Model Accuracy:", accuracy_score(y_test, y_pred_nonlinear))

Non-linear Model Accuracy: 0.8238003838771593


In [15]:
d = dice_ml.Data(dataframe=train_dataset,
                 continuous_features=['age', 'hours_per_week'],
                 outcome_name='income')

In [ ]:
m_logreg = dice_ml.Model(model=logreg, backend="sklearn")
exp_logreg = dice_ml.Dice(d, m_logreg, method="random")
e_logreg = exp_logreg.generate_counterfactuals(x_test[0:], total_CFs=4, desired_class="opposite")

In [ ]:
m_nonlinear = dice_ml.Model(model=nonlinear, backend="sklearn")
exp_nonlinear = dice_ml.Dice(d, m_nonlinear, method="random")
e_nonlinear = exp_nonlinear.generate_counterfactuals(x_test[0:], total_CFs=4, desired_class="opposite")

100%|██████████| 5210/5210 [22:49<00:00,  3.80it/s]


In [ ]:
for i, cf_exp in enumerate(e_logreg.cf_examples_list[:10]):
    print(f"Counterfactuals for query instance {i}:")
    cf_exp.visualize_as_dataframe(show_only_changes=True)

Counterfactuals for query instance 0:
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22,Private,HS-grad,Single,Blue-Collar,White,Male,16,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Prof-school,-,White-Collar,-,-,77,1
1,49,-,Doctorate,Married,-,-,-,-,1
2,87,-,-,Married,Professional,-,-,-,1
3,70,-,Doctorate,Married,-,-,-,-,1


Counterfactuals for query instance 1:
Query instance (original outcome : 0)


...

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,88,-,-,Married,-,-,-,63,1
1,-,-,-,Married,White-Collar,-,-,89,1
2,84,-,-,Married,-,-,-,-,1
3,82,-,-,Married,-,-,-,-,1


Counterfactuals for query instance 9:
Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50,Private,Some-college,Married,White-Collar,White,Male,45,1



Diverse Counterfactual set (new outcome: 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,34,-,-,-,Professional,-,-,-,0
1,-,-,-,-,-,-,-,22,0
2,-,-,-,Widowed,-,-,-,-,0
3,48,-,HS-grad,-,-,-,-,-,0


In [ ]:
for i, cf_exp in enumerate(e_nonlinear.cf_examples_list[:10]):
    print(f"Counterfactuals for query instance {i}:")
    cf_exp.visualize_as_dataframe(show_only_changes=True)

Counterfactuals for query instance 0:
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22,Private,HS-grad,Single,Blue-Collar,White,Male,16,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,63,-,Doctorate,-,-,-,-,-,1
1,63,-,Doctorate,Separated,-,-,-,-,1
2,54,-,Doctorate,Widowed,-,-,-,-,1
3,85,-,Prof-school,-,-,-,-,93,1


Counterfactuals for query instance 1:
Query instance (original outcome : 0)


...

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,56,-,Doctorate,-,-,-,-,-,1
1,55,-,Prof-school,Separated,-,-,-,-,1
2,85,-,Masters,-,-,-,-,98,1
3,-,-,Prof-school,Married,Professional,-,-,-,1


Counterfactuals for query instance 9:
Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50,Private,Some-college,Married,White-Collar,White,Male,45,1



Diverse Counterfactual set (new outcome: 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,82,-,-,Widowed,-,-,-,-,0
1,-,-,-,Divorced,-,-,-,-,0
2,18,-,-,Divorced,-,-,-,-,0
3,17,-,-,-,Blue-Collar,-,-,-,0


In [12]:
# save
# with open("e_linear.pkl", "wb") as f:
#     pickle.dump(e_logreg, f)

# with open("e_nonlinear.pkl", "wb") as f:
#     pickle.dump(e_nonlinear, f)

# load
with open("e_linear.pkl", "rb") as f:
    e_logreg = pickle.load(f)

with open("e_nonlinear.pkl", "rb") as f:
    e_nonlinear = pickle.load(f)

In [21]:
def analyze_and_print_cfs(model, model_name, cf_explainer, x_test, y_test, num_samples=5):
    y_pred = model.predict(x_test)

    sample_groups = {
        ('incorrect', 0): [],
        ('incorrect', 1): [],
        ('correct', 0): [],
        ('correct', 1): []
    }

    for idx in range(len(y_test)):
        true_label = y_test.iloc[idx]
        pred_label = y_pred[idx]

        key = ('correct' if true_label == pred_label else 'incorrect', true_label)
        if len(sample_groups[key]) < num_samples:
            sample_groups[key].append(idx)
            if all(len(v) >= num_samples for v in sample_groups.values()):
                break
    print(f"\n{'='*40}\nModel: {model_name}\n{'='*40}")

    categories = [
        ('Misclassified (True=0)', 'incorrect', 0),
        ('Misclassified (True=1)', 'incorrect', 1),
        ('Correctly Classified (True=0)', 'correct', 0),
        ('Correctly Classified (True=1)', 'correct', 1)
    ]

    for desc, group, label in categories:
        indices = sample_groups[(group, label)]
        print(f"\n{desc}:")
        for i, idx in enumerate(indices[:num_samples], 1):
            cf_exp = cf_explainer.cf_examples_list[idx]
            print(f"\nQuery instance {i} (Index {idx}):")
            print(f"True label: {label}, Predicted: {y_pred[idx]}")
            cf_exp.visualize_as_dataframe(show_only_changes=True)


In [55]:
print("="*60)
print("Analyzing Logistic Regression Model")
print("="*60)
analyze_and_print_cfs(logreg, "Logistic Regression", e_logreg, x_test, y_test)
print("\n\n" + "="*60)
print("Analyzing Neural Network Model")
print("="*60)
analyze_and_print_cfs(nonlinear, "Neural Network", e_nonlinear, x_test, y_test)

Analyzing Logistic Regression Model

Model: Logistic Regression

Misclassified (True=0):

Query instance 1 (Index 6):
True label: 0, Predicted: 1
Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,30,Private,Bachelors,Married,White-Collar,White,Female,40,1



Diverse Counterfactual set (new outcome: 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,-,Single,Professional,-,-,-,0
1,43,-,HS-grad,-,-,-,-,-,0
2,-,-,-,Separated,-,Other,-,-,0
3,-,Government,-,-,-,-,-,30,0



Query instance 2 (Index 30):
True label: 0, Predicted: 1
Query instance (original outcome : 1)


...

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,25,-,-,-,-,-,-,-,0
1,19,-,-,-,Blue-Collar,-,-,-,0
2,74,-,-,-,-,-,-,-,0
3,-,-,-,Separated,-,-,-,53,0



Query instance 5 (Index 26):
True label: 1, Predicted: 1
Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,61,Self-Employed,Bachelors,Married,White-Collar,White,Male,50,1



Diverse Counterfactual set (new outcome: 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,-,-,-,-,-,7,0
1,-,Government,-,Single,-,-,-,-,0
2,-,-,-,Divorced,-,-,-,-,0
3,18,-,-,-,-,Other,-,-,0


In [ ]:
def compute_mad(training_df, continuous_features):
    mad = {}
    for feat in continuous_features:
        vals = training_df[feat].astype(float).values
        median_val = np.median(vals)
        mad_val = np.median(np.abs(vals - median_val))
        if mad_val == 0:
            mad_val = 1e-6
        mad[feat] = mad_val
    return mad

def compute_cont_count_diversity(cf_df, continuous_features, threshold=1e-3):
    cf_vals = cf_df[continuous_features].astype(float).values
    n_cf = cf_vals.shape[0]
    count_diffs = []
    if n_cf <= 1:
        return 0.0
    for i in range(n_cf):
        for j in range(i+1, n_cf):
            diff_count = 0
            for idx_feat, feat in enumerate(continuous_features):
                if abs(cf_vals[i][idx_feat] - cf_vals[j][idx_feat]) > threshold:
                    diff_count += 1
            count_diffs.append(diff_count / len(continuous_features))
    return np.mean(count_diffs)

def evaluate_cf_metrics(query, cf_df, model, continuous_features, categorical_features, training_df, cont_count_threshold=1e-3):
    # 1. Validity
    orig_pred = model.predict(query)[0]
    cf_preds = model.predict(cf_df)
    validity = np.mean(cf_preds != orig_pred)

    mad = compute_mad(training_df, continuous_features)

    # 2. Continuous Proximity
    cont_dists = []
    for idx, row in cf_df.iterrows():
        norm_dists = []
        for feat in continuous_features:
            diff = abs(row[feat] - query.iloc[0][feat])
            norm_dists.append(diff / mad[feat])
        cont_dists.append(np.mean(norm_dists))
    cont_proximity = - np.mean(cont_dists)

    # 3. Categorical Proximity
    d_cat = len(categorical_features)
    cat_diffs = []
    for idx, row in cf_df.iterrows():
        diff_count = sum(1 for feat in categorical_features if row[feat] != query.iloc[0][feat])
        cat_diffs.append(diff_count / d_cat)
    cat_proximity = 1 - np.mean(cat_diffs)

    # 4. Continuous Diversity
    cf_cont = cf_df[continuous_features].astype(float).values
    cont_pair_dists = []
    n_cf = cf_cont.shape[0]
    if n_cf > 1:
        for i in range(n_cf):
            for j in range(i+1, n_cf):
                norm_dists = []
                for idx_feat, feat in enumerate(continuous_features):
                    diff = abs(cf_cont[i][idx_feat] - cf_cont[j][idx_feat])
                    norm_dists.append(diff / mad[feat])
                cont_pair_dists.append(np.mean(norm_dists))
        cont_diversity = np.mean(cont_pair_dists)
    else:
        cont_diversity = 0.0

    # 5. Categorical Diversity
    cat_pair_diffs = []
    n_cf = cf_df.shape[0]
    if n_cf > 1:
        for i in range(n_cf):
            for j in range(i+1, n_cf):
                diff_count = sum(1 for feat in categorical_features if cf_df.iloc[i][feat] != cf_df.iloc[j][feat])
                cat_pair_diffs.append(diff_count / d_cat)
        cat_diversity = np.mean(cat_pair_diffs)
    else:
        cat_diversity = 0.0

    # 6. Continuous Count Diversity
    cont_count_diversity = compute_cont_count_diversity(cf_df, continuous_features, threshold=cont_count_threshold)

    # 7. Sparsity
    all_feats = continuous_features + categorical_features
    d_total = len(all_feats)
    sparsity_vals = []
    for idx, row in cf_df.iterrows():
        diff_count = sum(1 for feat in all_feats if row[feat] != query.iloc[0][feat])
        sparsity_vals.append(diff_count / d_total)
    sparsity = np.mean(sparsity_vals)

    return {
        'validity': validity,
        'cont_proximity': cont_proximity,
        'cat_proximity': cat_proximity,
        'cont_diversity': cont_diversity,
        'cat_diversity': cat_diversity,
        'cont_count_diversity': cont_count_diversity,
        'sparsity': sparsity
    }

In [ ]:
continuous_features = ['age', 'hours_per_week']
all_predictors = x_train.columns.tolist()
categorical_features = list(set(all_predictors) - set(continuous_features))

logreg_validity_list = []
logreg_cont_prox_list = []
logreg_cat_prox_list = []
logreg_cont_div_list = []
logreg_cat_div_list = []
logreg_cont_count_div_list = []
logreg_sparsity_list = []

for i, cf_exp in enumerate(e_logreg.cf_examples_list):
    query_instance = x_test.iloc[[i]]
    cf_df = cf_exp.final_cfs_df
    metrics = evaluate_cf_metrics(query_instance, cf_df, logreg, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)

    logreg_validity_list.append(metrics['validity'])
    logreg_cont_prox_list.append(metrics['cont_proximity'])
    logreg_cat_prox_list.append(metrics['cat_proximity'])
    logreg_cont_div_list.append(metrics['cont_diversity'])
    logreg_cat_div_list.append(metrics['cat_diversity'])
    logreg_cont_count_div_list.append(metrics['cont_count_diversity'])
    logreg_sparsity_list.append(metrics['sparsity'])

logreg_avg_validity = np.mean(logreg_validity_list)
logreg_avg_cont_prox = np.mean(logreg_cont_prox_list)
logreg_avg_cat_prox = np.mean(logreg_cat_prox_list)
logreg_avg_cont_div = np.mean(logreg_cont_div_list)
logreg_avg_cat_div = np.mean(logreg_cat_div_list)
logreg_avg_cont_count_div = np.mean(logreg_cont_count_div_list)
logreg_avg_sparsity = np.mean(logreg_sparsity_list)

print("Aggregated Evaluation Metrics on the Test Set:")
print(f"Average Validity: {logreg_avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {logreg_avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {logreg_avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {logreg_avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {logreg_avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {logreg_avg_cont_count_div:.4f}")
print(f"Average Sparsity: {logreg_avg_sparsity:.4f}")


Aggregated Evaluation Metrics on the Test Set:
Average Validity: 100.00%
Average Continuous Proximity: -3.0761
Average Categorical Proximity: 0.8010
Average Continuous Diversity: 3.6581
Average Categorical Diversity: 0.2486
Average Continuous Count Diversity: 0.6295
Average Sparsity: 0.2527


In [ ]:
nonlinear_validity_list = []
nonlinear_cont_prox_list = []
nonlinear_cat_prox_list = []
nonlinear_cont_div_list = []
nonlinear_cat_div_list = []
nonlinear_cont_count_div_list = []
nonlinear_sparsity_list = []

for i, cf_exp in enumerate(e_nonlinear.cf_examples_list):
    query_instance = x_test.iloc[[i]]
    cf_df = cf_exp.final_cfs_df
    metrics = evaluate_cf_metrics(query_instance, cf_df, nonlinear, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)

    nonlinear_validity_list.append(metrics['validity'])
    nonlinear_cont_prox_list.append(metrics['cont_proximity'])
    nonlinear_cat_prox_list.append(metrics['cat_proximity'])
    nonlinear_cont_div_list.append(metrics['cont_diversity'])
    nonlinear_cat_div_list.append(metrics['cat_diversity'])
    nonlinear_cont_count_div_list.append(metrics['cont_count_diversity'])
    nonlinear_sparsity_list.append(metrics['sparsity'])

nonlinear_avg_validity = np.mean(nonlinear_validity_list)
nonlinear_avg_cont_prox = np.mean(nonlinear_cont_prox_list)
nonlinear_avg_cat_prox = np.mean(nonlinear_cat_prox_list)
nonlinear_avg_cont_div = np.mean(nonlinear_cont_div_list)
nonlinear_avg_cat_div = np.mean(nonlinear_cat_div_list)
nonlinear_avg_cont_count_div = np.mean(nonlinear_cont_count_div_list)
nonlinear_avg_sparsity = np.mean(nonlinear_sparsity_list)

print("Aggregated Evaluation Metrics on the Test Set:")
print(f"Average Validity: {nonlinear_avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {nonlinear_avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {nonlinear_avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {nonlinear_avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {nonlinear_avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {nonlinear_avg_cont_count_div:.4f}")
print(f"Average Sparsity: {nonlinear_avg_sparsity:.4f}")

Aggregated Evaluation Metrics on the Test Set:
Average Validity: 100.00%
Average Continuous Proximity: -2.5579
Average Categorical Proximity: 0.7942
Average Continuous Diversity: 2.9823
Average Categorical Diversity: 0.2728
Average Continuous Count Diversity: 0.5869
Average Sparsity: 0.2545


#reimplementation from scratch

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import time
from tqdm import tqdm

class PyTorchDiCE:
    def __init__(self, model, continuous_features, categorical_features,
                 data_for_mad, lambda1=0.5, lambda2=1.0, lr=0.05, max_iter=5000):
        self.model = model
        self.continuous_features = continuous_features
        self.categorical_features = categorical_features
        self.all_features = continuous_features + categorical_features
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lr = lr
        self.max_iter = max_iter

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        self.mad = self._compute_mad(data_for_mad)
        self._setup_categorical_encodings(data_for_mad)
        self._create_surrogate_model(model, data_for_mad)

    def _compute_mad(self, data):
        mad = {}
        for feat in self.continuous_features:
            values = data[feat].values
            median = np.median(values)
            mad[feat] = np.median(np.abs(values - median))
            if mad[feat] == 0:
                mad[feat] = 1e-6
        return mad

    def _setup_categorical_encodings(self, data):
        self.cat_encodings = {}
        self.cat_dims = {}
        self.cat_values = {}

        for feat in self.categorical_features:
            unique_values = data[feat].unique().tolist()
            self.cat_values[feat] = unique_values
            self.cat_dims[feat] = len(unique_values)
            self.cat_encodings[feat] = {val: i for i, val in enumerate(unique_values)}

    def _encode_instance(self, instance):
        encoded = []
        for feat in self.continuous_features:
            encoded.append(float(instance[feat]))

        for feat in self.categorical_features:
            cat_value = instance[feat]
            one_hot = [0.0] * self.cat_dims[feat]
            if cat_value in self.cat_encodings[feat]:
                one_hot[self.cat_encodings[feat][cat_value]] = 1.0
            encoded.extend(one_hot)

        encoded_tensor = torch.tensor(encoded, dtype=torch.float32, device=self.device)

        return encoded_tensor

    def _decode_tensor(self, tensor):
        tensor_np = tensor.detach().cpu().numpy()
        instance = {}

        idx = 0
        for feat in self.continuous_features:
            instance[feat] = tensor_np[idx]
            idx += 1

        for feat in self.categorical_features:
            one_hot = tensor_np[idx:idx+self.cat_dims[feat]]
            cat_idx = np.argmax(one_hot)
            instance[feat] = self.cat_values[feat][cat_idx]
            idx += self.cat_dims[feat]

        return instance

    def _create_surrogate_model(self, original_model, data):
        sample_encoded = self._encode_instance(data.iloc[0].to_dict())
        input_dim = len(sample_encoded)

        class SurrogateModel(nn.Module):
            def __init__(self, input_dim):
                super().__init__()
                self.model = nn.Sequential(
                    nn.Linear(input_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, 16),
                    nn.ReLU(),
                    nn.Linear(16, 1),
                    nn.Sigmoid()
                )

            def forward(self, x):
                return self.model(x)

        self.surrogate = SurrogateModel(input_dim).to(self.device)
        print("Training surrogate model...")
        self._train_surrogate(original_model, data)

    def _train_surrogate(self, original_model, data, epochs=50, batch_size=128):
        optimizer = torch.optim.Adam(self.surrogate.parameters(), lr=0.001)
        criterion = nn.BCELoss()
        original_preds = original_model.predict_proba(data)[:, 1]

        encoded_data = []
        for _, row in data.iterrows():
            encoded_data.append(self._encode_instance(row.to_dict()))

        dataset = torch.utils.data.TensorDataset(
            torch.stack(encoded_data),
            torch.tensor(original_preds, dtype=torch.float32, device=self.device)
        )
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.surrogate.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch_x, batch_y in dataloader:
                optimizer.zero_grad()
                outputs = self.surrogate(batch_x).squeeze()
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.6f}")

        self.surrogate.eval()
        with torch.no_grad():
            surrogate_preds = []
            for batch_x, _ in dataloader:
                outputs = self.surrogate(batch_x).squeeze().cpu().numpy()
                surrogate_preds.extend(outputs)

            original_preds = original_model.predict_proba(data)[:, 1]
            mse = np.mean((np.array(surrogate_preds) - original_preds) ** 2)
            print(f"Surrogate model MSE: {mse:.6f}")

    def _hinge_loss(self, pred_prob, desired_class):
        z = 1.0 if desired_class == 1 else -1.0
        eps = 1e-6
        logit = torch.log(pred_prob / (1 - pred_prob + eps) + eps)
        return torch.relu(1.0 - z * logit)

    def _compute_diversity_loss(self, encoded_cfs):
        batch_size = encoded_cfs.shape[0]
        if batch_size <= 1:
            return torch.tensor(0.0, device=self.device)

        distances = torch.cdist(encoded_cfs, encoded_cfs, p=2)

        kernel = 1.0 / (1.0 + distances + torch.eye(batch_size, device=self.device) * 1e-6)

        diag_noise = torch.rand(batch_size, device=self.device) * 1e-6
        kernel = kernel + torch.diag(diag_noise)
        diversity = torch.logdet(kernel)

        return -diversity

    def _proximity_loss(self, encoded_cfs, encoded_query):
        diffs = torch.abs(encoded_cfs - encoded_query)

        mad_values = []
        for feat in self.continuous_features:
            mad_values.append(1.0 / self.mad[feat])

        idx = len(self.continuous_features)
        for feat in self.categorical_features:
            mad_values.extend([1.0] * self.cat_dims[feat])

        mad_tensor = torch.tensor(mad_values, dtype=torch.float32, device=self.device)

        scaled_diffs = diffs * mad_tensor

        return torch.mean(scaled_diffs)

    def generate_counterfactuals(self, query_instance, desired_class=None, k=4):
        if isinstance(query_instance, pd.DataFrame):
            if len(query_instance) == 1:
                query_instance = query_instance.iloc[0].to_dict()

        elif isinstance(query_instance, pd.Series):
            query_instance = query_instance.to_dict()
        if desired_class is None:
            query_df = pd.DataFrame([query_instance])
            pred_class = 1 if self.model.predict_proba(query_df)[0][1] > 0.5 else 0
            desired_class = 1 - pred_class

        encoded_query = self._encode_instance(query_instance)

        cf_tensors = []
        for i in range(k):
            cf_tensor = encoded_query.clone()

            for j in range(len(self.continuous_features)):
                noise = torch.randn(1, device=self.device) * 0.7 * self.mad[self.continuous_features[j]]
                cf_tensor[j] += noise.item()

            idx = len(self.continuous_features)
            for feat in self.categorical_features:
                if torch.rand(1, device=self.device) < 0.25:
                    cf_tensor[idx:idx+self.cat_dims[feat]] = 0.0
                    new_cat = torch.randint(0, self.cat_dims[feat], (1,), device=self.device)
                    cf_tensor[idx + new_cat] = 1.0
                idx += self.cat_dims[feat]

            cf_tensors.append(cf_tensor)

        cf_param = torch.stack(cf_tensors).clone().detach().requires_grad_(True)

        optimizer = torch.optim.Adam([cf_param], lr=self.lr)

        for iteration in range(self.max_iter):
            optimizer.zero_grad()
            processed_cf = cf_param.clone()
            idx = len(self.continuous_features)
            for feat in self.categorical_features:
                cat_logits = processed_cf[:, idx:idx+self.cat_dims[feat]]
                cat_probs = torch.softmax(cat_logits * 10.0, dim=1)
                processed_cf[:, idx:idx+self.cat_dims[feat]] = cat_probs
                idx += self.cat_dims[feat]

            pred_probs = self.surrogate(processed_cf).squeeze()
            y_loss = torch.mean(self._hinge_loss(pred_probs, desired_class))
            prox_loss = self._proximity_loss(processed_cf, encoded_query)
            div_loss = self._compute_diversity_loss(processed_cf)
            total_loss = y_loss + self.lambda1 * prox_loss + self.lambda2 * div_loss
            total_loss.backward()
            optimizer.step()

            if iteration % 500 == 0 or iteration == self.max_iter - 1:
                with torch.no_grad():
                    valid_cfs = torch.sum((pred_probs > 0.5) == (desired_class == 1)).item()
                    # print(f"Iteration {iteration}, Loss: {total_loss.item():.4f}, "
                          # f"Y-Loss: {y_loss.item():.4f}, Valid CFs: {valid_cfs}/{k}")

                if valid_cfs == k and iteration > 1000:
                    # print(f"Early stopping at iteration {iteration}")
                    break

        with torch.no_grad():
            processed_cf = cf_param.clone()
            idx = len(self.continuous_features)
            for feat in self.categorical_features:
                cat_logits = processed_cf[:, idx:idx+self.cat_dims[feat]]
                cat_indices = torch.argmax(cat_logits, dim=1)
                processed_cf[:, idx:idx+self.cat_dims[feat]] = 0.0
                for i, cat_idx in enumerate(cat_indices):
                    processed_cf[i, idx + cat_idx] = 1.0
                idx += self.cat_dims[feat]

        counterfactuals = []
        for i in range(k):
            cf_dict = self._decode_tensor(processed_cf[i])
            counterfactuals.append(cf_dict)

        return counterfactuals


    def generate_counterfactuals_for_dataset(self, data, desired_class="opposite", k=4, batch_size=5):
        all_counterfactuals = []
        total_instances = len(data)


        pbar = tqdm(total=total_instances, desc="Generating CFs for dataset")
        for i in range(0, total_instances, batch_size):
            batch_end = min(i + batch_size, total_instances)
            batch = data.iloc[i:batch_end]
            batch_counterfactuals = []
            for _, row in batch.iterrows():
                query_instance = row.to_dict()

                if desired_class == "opposite":
                    query_df = pd.DataFrame([query_instance])
                    pred_class = 1 if self.model.predict_proba(query_df)[0][1] > 0.5 else 0
                    target_class = 1 - pred_class
                else:
                    target_class = desired_class

                counterfactuals = self.generate_counterfactuals(
                    query_instance, target_class, k)
                batch_counterfactuals.append(counterfactuals)
                pbar.update(1)

            all_counterfactuals.extend(batch_counterfactuals)

        return all_counterfactuals

In [ ]:
pytorch_dice = PyTorchDiCE(
    model=logreg,
    continuous_features=continuous_features,
    categorical_features=categorical_features,
    data_for_mad=x_train,
    lambda1=0.5,
    lambda2=1.0,
    lr=0.05,
    max_iter=5000
)

Using device: cpu
Training surrogate model...
Epoch 10/50, Loss: 0.366261
Epoch 20/50, Loss: 0.366297
Epoch 30/50, Loss: 0.366180
Epoch 40/50, Loss: 0.366211
Epoch 50/50, Loss: 0.366173
Surrogate model MSE: 0.129967


In [ ]:
test_samples = x_test[0:100]

cf_results_pytorch = pytorch_dice.generate_counterfactuals_for_dataset(
    test_samples,
    desired_class="opposite",
    k=4,
    batch_size=5
)

class CFResult:
    def __init__(self, query, cfs):
        self.original_instance = query
        self.final_cfs_df = pd.DataFrame(cfs)

class DiceResults:
    def __init__(self, cf_results_list):
        self.cf_examples_list = cf_results_list

cf_examples_list = []
for i, (_, query) in enumerate(test_samples.iterrows()):
    cfs = cf_results_pytorch[i]
    cf_examples_list.append(CFResult(query, cfs))

e1 = DiceResults(cf_examples_list)

validity_list = []
cont_prox_list = []
cat_prox_list = []
cont_div_list = []
cat_div_list = []
cont_count_div_list = []
sparsity_list = []

for i, cf_exp in enumerate(e1.cf_examples_list):
    query_instance = test_samples.iloc[[i]]
    cf_df = cf_exp.final_cfs_df
    metrics = evaluate_cf_metrics(query_instance, cf_df, logreg, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)

    validity_list.append(metrics['validity'])
    cont_prox_list.append(metrics['cont_proximity'])
    cat_prox_list.append(metrics['cat_proximity'])
    cont_div_list.append(metrics['cont_diversity'])
    cat_div_list.append(metrics['cat_diversity'])
    cont_count_div_list.append(metrics['cont_count_diversity'])
    sparsity_list.append(metrics['sparsity'])

avg_validity = np.mean(validity_list)
avg_cont_prox = np.mean(cont_prox_list)
avg_cat_prox = np.mean(cat_prox_list)
avg_cont_div = np.mean(cont_div_list)
avg_cat_div = np.mean(cat_div_list)
avg_cont_count_div = np.mean(cont_count_div_list)
avg_sparsity = np.mean(sparsity_list)

print("\nPyTorch DiCE Evaluation Results:")
print(f"Average Validity: {avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {avg_cont_count_div:.4f}")
print(f"Average Sparsity: {avg_sparsity:.4f}")

In [ ]:
with open("re_linear.pkl", "wb") as f:
    pickle.dump(e1, f)

In [ ]:
pytorch_dice_nonlinear = PyTorchDiCE(
    model=nonlinear,
    continuous_features=continuous_features,
    categorical_features=categorical_features,
    data_for_mad=x_train,
    lambda1=0.5,
    lambda2=1.0,
    lr=0.05,
    max_iter=5000
)

Using device: cpu
Training surrogate model...
Epoch 10/50, Loss: 0.355020
Epoch 20/50, Loss: 0.350602
Epoch 30/50, Loss: 0.349316
Epoch 40/50, Loss: 0.348692
Epoch 50/50, Loss: 0.348334
Surrogate model MSE: 0.143268


In [ ]:
print("Generating counterfactuals nonlinear")
cf_results_pytorch = pytorch_dice_nonlinear.generate_counterfactuals_for_dataset(
    test_samples,
    desired_class="opposite",
    k=4,
    batch_size=5
)

cf_examples_list = []
for i, (_, query) in enumerate(test_samples.iterrows()):
    cfs = cf_results_pytorch[i]
    cf_examples_list.append(CFResult(query, cfs))

e1_nonlinear = DiceResults(cf_examples_list)

validity_list = []
cont_prox_list = []
cat_prox_list = []
cont_div_list = []
cat_div_list = []
cont_count_div_list = []
sparsity_list = []

for i, cf_exp in enumerate(e1_nonlinear.cf_examples_list):
    query_instance = test_samples.iloc[[i]]
    cf_df = cf_exp.final_cfs_df

    metrics = evaluate_cf_metrics(query_instance, cf_df, nonlinear, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)

    validity_list.append(metrics['validity'])
    cont_prox_list.append(metrics['cont_proximity'])
    cat_prox_list.append(metrics['cat_proximity'])
    cont_div_list.append(metrics['cont_diversity'])
    cat_div_list.append(metrics['cat_diversity'])
    cont_count_div_list.append(metrics['cont_count_diversity'])
    sparsity_list.append(metrics['sparsity'])

avg_validity = np.mean(validity_list)
avg_cont_prox = np.mean(cont_prox_list)
avg_cat_prox = np.mean(cat_prox_list)
avg_cont_div = np.mean(cont_div_list)
avg_cat_div = np.mean(cat_div_list)
avg_cont_count_div = np.mean(cont_count_div_list)
avg_sparsity = np.mean(sparsity_list)

print("\nPyTorch DiCE nonlinear Evaluation Results:")
print(f"Average Validity: {avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {avg_cont_count_div:.4f}")
print(f"Average Sparsity: {avg_sparsity:.4f}")

In [ ]:
with open("re_nonlinear.pkl", "wb") as f:
    pickle.dump(e1_nonlinear, f)

#Feasibility

##linear

In [38]:
with open("e_linear.pkl", "rb") as f:
    e_logreg = pickle.load(f)

In [ ]:
filtered_e_logreg, filter_stats = filter_counterfactuals_with_stats(e_logreg, desired_cfs_per_instance=4)

for i, cf_exp in enumerate(filtered_e_logreg.cf_examples_list[:10]):
    print(f"Counterfactuals for query instance {i}:")
    cf_exp.visualize_as_dataframe(show_only_changes=True)

In [27]:
continuous_features = ['age', 'hours_per_week']
all_predictors = x_train.columns.tolist()  # predictors from training set
categorical_features = list(set(all_predictors) - set(continuous_features))

# Initialize lists to store metrics for each query instance.
logreg_validity_list = []
logreg_cont_prox_list = []
logreg_cat_prox_list = []
logreg_cont_div_list = []
logreg_cat_div_list = []
logreg_cont_count_div_list = []
logreg_sparsity_list = []

num_with_valid_cfs = 0
num_total = len(filtered_e_logreg.cf_examples_list)

# Loop over each query instance for which CFs were generated.
for i, cf_exp in enumerate(filtered_e_logreg.cf_examples_list):
    # Get the i-th query instance (as one-row DataFrame)
    query_instance = x_test.iloc[[i]]
    # Get the counterfactuals for this query.
    cf_df = cf_exp.final_cfs_df
    # Evaluate metrics.
    metrics = evaluate_cf_metrics(query_instance, cf_df, logreg, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)


    if metrics is not None:
        num_with_valid_cfs += 1
        logreg_validity_list.append(metrics['validity'])
        logreg_cont_prox_list.append(metrics['cont_proximity'])
        logreg_cat_prox_list.append(metrics['cat_proximity'])
        logreg_cont_div_list.append(metrics['cont_diversity'])
        logreg_cat_div_list.append(metrics['cat_diversity'])
        logreg_cont_count_div_list.append(metrics['cont_count_diversity'])
        logreg_sparsity_list.append(metrics['sparsity'])

logreg_avg_validity = np.mean(logreg_validity_list)
logreg_avg_cont_prox = np.mean(logreg_cont_prox_list)
logreg_avg_cat_prox = np.mean(logreg_cat_prox_list)
logreg_avg_cont_div = np.mean(logreg_cont_div_list)
logreg_avg_cat_div = np.mean(logreg_cat_div_list)
logreg_avg_cont_count_div = np.mean(logreg_cont_count_div_list)
logreg_avg_sparsity = np.mean(logreg_sparsity_list)

print(f"Instances with valid CFs after filtering: {num_with_valid_cfs}/{num_total} ({100*num_with_valid_cfs/num_total:.2f}%)")
print(f"Average Validity: {logreg_avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {logreg_avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {logreg_avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {logreg_avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {logreg_avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {logreg_avg_cont_count_div:.4f}")
print(f"Average Sparsity: {logreg_avg_sparsity:.4f}")

Instances with valid CFs after filtering: 5084/5210 (97.58%)
Average Validity: 100.00%
Average Continuous Proximity: -3.5378
Average Categorical Proximity: 0.8310
Average Continuous Diversity: 3.1528
Average Categorical Diversity: 0.1763
Average Continuous Count Diversity: 0.5840
Average Sparsity: 0.2470


In [30]:
with open("feasibility_linear.pkl", "wb") as f:
    pickle.dump(filtered_e_logreg, f)

In [37]:
import copy

def filter_counterfactuals_with_stats(cf_examples, desired_cfs_per_instance=4):
    filtered_cf_examples = copy.deepcopy(cf_examples)

    education_order = {
        "HS-grad": 0,
        "Some-college": 1,
        "Associates": 2,
        "Bachelors": 3,
        "Masters": 4,
        "Prof-school": 5,
        "Doctorate": 6
    }

    stats = {
        'total_original_cfs': 0,
        'total_filtered_cfs': 0,
        'income_0_to_1': {
            'original_cfs': 0,
            'filtered_cfs': 0,
            'reasons': {
                'race_changed': 0,
                'race_changes': {},
                'gender_changed': 0,
                'gender_changes': {},
                'age_decreased': 0,
                'education_decreased': 0,
                'education_age_mismatch': 0,
                'valid_cfs': 0
            }
        },
        'income_1_to_0': {
            'original_cfs': 0,
            'filtered_cfs': 0,
            'reasons': {
                'race_changed': 0,
                'race_changes': {},
                'gender_changed': 0,
                'gender_changes': {},
                'age_decreased': 0,
                'education_decreased': 0,
                'education_age_mismatch': 0,
                'valid_cfs': 0
            }
        },
        'per_instance': []
    }

    for i, cf_list in enumerate(filtered_cf_examples.cf_examples_list):
        original_instance = cf_list.test_instance_df.iloc[0].to_dict()
        all_cfs = cf_list.final_cfs_df

        original_income = 0
        if 'income' in original_instance:
            if isinstance(original_instance['income'], str):
                original_income = 1 if original_instance['income'] == '>50K' else 0
            else:
                original_income = original_instance['income']

        income_direction = 'income_0_to_1' if original_income == 0 else 'income_1_to_0'

        if all_cfs.empty:
            stats['per_instance'].append({
                'instance_idx': i,
                'original_income': original_income,
                'income_direction': income_direction,
                'original_cfs': 0,
                'filtered_cfs': 0,
                'reasons': {}
            })
            continue

        instance_stats = {
            'instance_idx': i,
            'original_income': original_income,
            'income_direction': income_direction,
            'original_cfs': len(all_cfs),
            'filtered_cfs': 0,
            'reasons': {
                'race_changed': 0,
                'gender_changed': 0,
                'age_decreased': 0,
                'education_decreased': 0,
                'education_age_mismatch': 0
            },
            'race_changes': {},
            'gender_changes': {}
        }

        stats['total_original_cfs'] += len(all_cfs)
        stats[income_direction]['original_cfs'] += len(all_cfs)

        valid_indices = []

        for j in range(len(all_cfs)):
            cf = all_cfs.iloc[j].to_dict()
            is_valid = True
            reasons = []

            if 'race' in cf and 'race' in original_instance:
                if cf['race'] != original_instance['race']:
                    is_valid = False
                    reasons.append('race_changed')
                    instance_stats['reasons']['race_changed'] += 1
                    stats[income_direction]['reasons']['race_changed'] += 1

                    race_change = f"{original_instance['race']}->{cf['race']}"
                    instance_stats['race_changes'][race_change] = instance_stats['race_changes'].get(race_change, 0) + 1
                    stats[income_direction]['reasons']['race_changes'][race_change] = stats[income_direction]['reasons']['race_changes'].get(race_change, 0) + 1

            if 'gender' in cf and 'gender' in original_instance:
                if cf['gender'] != original_instance['gender']:
                    is_valid = False
                    reasons.append('gender_changed')
                    instance_stats['reasons']['gender_changed'] += 1
                    stats[income_direction]['reasons']['gender_changed'] += 1

                    gender_change = f"{original_instance['gender']}->{cf['gender']}"
                    instance_stats['gender_changes'][gender_change] = instance_stats['gender_changes'].get(gender_change, 0) + 1
                    stats[income_direction]['reasons']['gender_changes'][gender_change] = stats[income_direction]['reasons']['gender_changes'].get(gender_change, 0) + 1

            if 'age' in cf and 'age' in original_instance:
                if cf['age'] < original_instance['age']:
                    is_valid = False
                    reasons.append('age_decreased')
                    instance_stats['reasons']['age_decreased'] += 1
                    stats[income_direction]['reasons']['age_decreased'] += 1

            if 'education' in cf and 'education' in original_instance:
                if (cf['education'] in education_order and
                    original_instance['education'] in education_order):
                    cf_edu_level = education_order.get(cf['education'], -1)
                    orig_edu_level = education_order.get(original_instance['education'], -1)

                    if cf_edu_level < orig_edu_level:
                        is_valid = False
                        reasons.append('education_decreased')
                        instance_stats['reasons']['education_decreased'] += 1
                        stats[income_direction]['reasons']['education_decreased'] += 1

                    if cf_edu_level > orig_edu_level:
                        min_age_increase = (cf_edu_level - orig_edu_level) * 1
                        if cf['age'] < original_instance['age'] + min_age_increase:
                            is_valid = False
                            reasons.append('education_age_mismatch')
                            instance_stats['reasons']['education_age_mismatch'] += 1
                            stats[income_direction]['reasons']['education_age_mismatch'] += 1

            if is_valid:
                valid_indices.append(j)

        if valid_indices:
            valid_indices = valid_indices[:desired_cfs_per_instance]
            instance_stats['filtered_cfs'] = len(valid_indices)
            stats[income_direction]['reasons']['valid_cfs'] += len(valid_indices)
            stats[income_direction]['filtered_cfs'] += len(valid_indices)
            stats['total_filtered_cfs'] += len(valid_indices)

            cf_list.final_cfs_df = all_cfs.iloc[valid_indices].reset_index(drop=True)

            if hasattr(cf_list, 'final_cfs_list') and cf_list.final_cfs_list is not None:
                if isinstance(cf_list.final_cfs_list, list) and len(cf_list.final_cfs_list) > 0:
                    cf_list.final_cfs_list = [cf_list.final_cfs_list[idx] for idx in valid_indices]
                else:
                    cf_list.final_cfs_list = []

            if hasattr(cf_list, 'final_cfs'):
                if cf_list.final_cfs is not None:
                    if isinstance(cf_list.final_cfs, list):
                        if len(cf_list.final_cfs) > 0:
                            cf_list.final_cfs = [cf_list.final_cfs[idx] for idx in valid_indices]
                        else:
                            cf_list.final_cfs = []
                    elif hasattr(cf_list.final_cfs, 'iloc'):
                        cf_list.final_cfs = cf_list.final_cfs.iloc[valid_indices].reset_index(drop=True)
                    else:
                        cf_list.final_cfs = cf_list.final_cfs_df

            cf_list.total_CFs = len(valid_indices)
        else:
            instance_stats['filtered_cfs'] = 0
            cf_list.final_cfs_df = pd.DataFrame()
            if hasattr(cf_list, 'final_cfs_list'):
                cf_list.final_cfs_list = []
            if hasattr(cf_list, 'final_cfs'):
                if isinstance(cf_list.final_cfs, list):
                    cf_list.final_cfs = []
                else:
                    cf_list.final_cfs = pd.DataFrame()
            cf_list.total_CFs = 0

        stats['per_instance'].append(instance_stats)

    for direction in ['income_0_to_1', 'income_1_to_0']:
        if stats[direction]['original_cfs'] > 0:
            stats[direction]['percentage_filtered'] = 100 * (stats[direction]['original_cfs'] - stats[direction]['filtered_cfs']) / stats[direction]['original_cfs']

            for reason in ['race_changed', 'gender_changed', 'age_decreased', 'education_decreased', 'education_age_mismatch']:
                stats[direction][f'{reason}_percentage'] = 100 * stats[direction]['reasons'][reason] / stats[direction]['original_cfs']

    if stats['total_original_cfs'] > 0:
        stats['percentage_filtered'] = 100 * (stats['total_original_cfs'] - stats['total_filtered_cfs']) / stats['total_original_cfs']

    return filtered_cf_examples, stats

In [39]:
filtered_e_logreg2, filter_stats = filter_counterfactuals_with_stats(e_logreg, desired_cfs_per_instance=4)

print(f"Original counterfactuals: {filter_stats['total_original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats['total_filtered_cfs']}")
print(f"Percentage filtered: {filter_stats['percentage_filtered']:.2f}%\n")

print("=== Income 0 to 1 (<=50K to >50K) ===")
direction = 'income_0_to_1'
print(f"Original counterfactuals: {filter_stats[direction]['original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats[direction]['filtered_cfs']}")
if filter_stats[direction]['original_cfs'] > 0:
    print(f"Percentage filtered: {filter_stats[direction]['percentage_filtered']:.2f}%")

    print("\nReasons for filtering:")
    print(f"Race changed: {filter_stats[direction]['reasons']['race_changed']} ({filter_stats[direction]['race_changed_percentage']:.2f}%)")
    print(f"Gender changed: {filter_stats[direction]['reasons']['gender_changed']} ({filter_stats[direction]['gender_changed_percentage']:.2f}%)")
    print(f"Age decreased: {filter_stats[direction]['reasons']['age_decreased']} ({filter_stats[direction]['age_decreased_percentage']:.2f}%)")
    print(f"Education decreased: {filter_stats[direction]['reasons']['education_decreased']} ({filter_stats[direction]['education_decreased_percentage']:.2f}%)")
    print(f"Education-age mismatch: {filter_stats[direction]['reasons']['education_age_mismatch']} ({filter_stats[direction]['education_age_mismatch_percentage']:.2f}%)")
    print(f"Valid CFs: {filter_stats[direction]['reasons']['valid_cfs']}")

    print("\nRace change details:")
    for change, count in filter_stats[direction]['reasons']['race_changes'].items():
        print(f"  {change}: {count}")

    print("\nGender change details:")
    for change, count in filter_stats[direction]['reasons']['gender_changes'].items():
        print(f"  {change}: {count}")

print("\n=== Income 1 to 0 (>50K to <=50K) ===")
direction = 'income_1_to_0'
print(f"Original counterfactuals: {filter_stats[direction]['original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats[direction]['filtered_cfs']}")
if filter_stats[direction]['original_cfs'] > 0:
    print(f"Percentage filtered: {filter_stats[direction]['percentage_filtered']:.2f}%")

    print("\nReasons for filtering:")
    print(f"Race changed: {filter_stats[direction]['reasons']['race_changed']} ({filter_stats[direction]['race_changed_percentage']:.2f}%)")
    print(f"Gender changed: {filter_stats[direction]['reasons']['gender_changed']} ({filter_stats[direction]['gender_changed_percentage']:.2f}%)")
    print(f"Age decreased: {filter_stats[direction]['reasons']['age_decreased']} ({filter_stats[direction]['age_decreased_percentage']:.2f}%)")
    print(f"Education decreased: {filter_stats[direction]['reasons']['education_decreased']} ({filter_stats[direction]['education_decreased_percentage']:.2f}%)")
    print(f"Education-age mismatch: {filter_stats[direction]['reasons']['education_age_mismatch']} ({filter_stats[direction]['education_age_mismatch_percentage']:.2f}%)")
    print(f"Valid CFs: {filter_stats[direction]['reasons']['valid_cfs']}")

    print("\nRace change details:")
    for change, count in filter_stats[direction]['reasons']['race_changes'].items():
        print(f"  {change}: {count}")

    print("\nGender change details:")
    for change, count in filter_stats[direction]['reasons']['gender_changes'].items():
        print(f"  {change}: {count}")

Original counterfactuals: 20840
Filtered counterfactuals: 13919
Percentage filtered: 33.21%

=== Income 0 to 1 (<=50K to >50K) ===
Original counterfactuals: 17028
Filtered counterfactuals: 11722
Percentage filtered: 31.16%

Reasons for filtering:
Race changed: 501 (2.94%)
Gender changed: 718 (4.22%)
Age decreased: 342 (2.01%)
Education decreased: 24 (0.14%)
Education-age mismatch: 4140 (24.31%)
Valid CFs: 11722

Race change details:
  White->Other: 359
  Other->White: 142

Gender change details:
  Female->Male: 443
  Male->Female: 275

=== Income 1 to 0 (>50K to <=50K) ===
Original counterfactuals: 3812
Filtered counterfactuals: 2197
Percentage filtered: 42.37%

Reasons for filtering:
Race changed: 278 (7.29%)
Gender changed: 298 (7.82%)
Age decreased: 676 (17.73%)
Education decreased: 479 (12.57%)
Education-age mismatch: 67 (1.76%)
Valid CFs: 2197

Race change details:
  White->Other: 264
  Other->White: 14

Gender change details:
  Male->Female: 290
  Female->Male: 8


##nonlinear

In [40]:
with open("e_nonlinear.pkl", "rb") as f:
    e_nonlinear = pickle.load(f)

In [ ]:
filtered_e_nonlinear, filter_stats = filter_counterfactuals_with_stats(e_nonlinear, desired_cfs_per_instance=4)

for i, cf_exp in enumerate(filtered_e_nonlinear.cf_examples_list[:10]):
    print(f"Counterfactuals for query instance {i}:")
    cf_exp.visualize_as_dataframe(show_only_changes=True)

In [41]:
filtered_e_nonlinear2, filter_stats = filter_counterfactuals_with_stats(e_nonlinear, desired_cfs_per_instance=4)
print(f"Original counterfactuals: {filter_stats['total_original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats['total_filtered_cfs']}")
print(f"Percentage filtered: {filter_stats['percentage_filtered']:.2f}%\n")

print("=== Income 0 to 1 (<=50K to >50K) ===")
direction = 'income_0_to_1'
print(f"Original counterfactuals: {filter_stats[direction]['original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats[direction]['filtered_cfs']}")
if filter_stats[direction]['original_cfs'] > 0:
    print(f"Percentage filtered: {filter_stats[direction]['percentage_filtered']:.2f}%")

    print("\nReasons for filtering:")
    print(f"Race changed: {filter_stats[direction]['reasons']['race_changed']} ({filter_stats[direction]['race_changed_percentage']:.2f}%)")
    print(f"Gender changed: {filter_stats[direction]['reasons']['gender_changed']} ({filter_stats[direction]['gender_changed_percentage']:.2f}%)")
    print(f"Age decreased: {filter_stats[direction]['reasons']['age_decreased']} ({filter_stats[direction]['age_decreased_percentage']:.2f}%)")
    print(f"Education decreased: {filter_stats[direction]['reasons']['education_decreased']} ({filter_stats[direction]['education_decreased_percentage']:.2f}%)")
    print(f"Education-age mismatch: {filter_stats[direction]['reasons']['education_age_mismatch']} ({filter_stats[direction]['education_age_mismatch_percentage']:.2f}%)")
    print(f"Valid CFs: {filter_stats[direction]['reasons']['valid_cfs']}")

    print("\nRace change details:")
    for change, count in filter_stats[direction]['reasons']['race_changes'].items():
        print(f"  {change}: {count}")

    print("\nGender change details:")
    for change, count in filter_stats[direction]['reasons']['gender_changes'].items():
        print(f"  {change}: {count}")

print("\n=== Income 1 to 0 (>50K to <=50K) ===")
direction = 'income_1_to_0'
print(f"Original counterfactuals: {filter_stats[direction]['original_cfs']}")
print(f"Filtered counterfactuals: {filter_stats[direction]['filtered_cfs']}")
if filter_stats[direction]['original_cfs'] > 0:
    print(f"Percentage filtered: {filter_stats[direction]['percentage_filtered']:.2f}%")

    print("\nReasons for filtering:")
    print(f"Race changed: {filter_stats[direction]['reasons']['race_changed']} ({filter_stats[direction]['race_changed_percentage']:.2f}%)")
    print(f"Gender changed: {filter_stats[direction]['reasons']['gender_changed']} ({filter_stats[direction]['gender_changed_percentage']:.2f}%)")
    print(f"Age decreased: {filter_stats[direction]['reasons']['age_decreased']} ({filter_stats[direction]['age_decreased_percentage']:.2f}%)")
    print(f"Education decreased: {filter_stats[direction]['reasons']['education_decreased']} ({filter_stats[direction]['education_decreased_percentage']:.2f}%)")
    print(f"Education-age mismatch: {filter_stats[direction]['reasons']['education_age_mismatch']} ({filter_stats[direction]['education_age_mismatch_percentage']:.2f}%)")
    print(f"Valid CFs: {filter_stats[direction]['reasons']['valid_cfs']}")

    print("\nRace change details:")
    for change, count in filter_stats[direction]['reasons']['race_changes'].items():
        print(f"  {change}: {count}")

    print("\nGender change details:")
    for change, count in filter_stats[direction]['reasons']['gender_changes'].items():
        print(f"  {change}: {count}")

Original counterfactuals: 20840
Filtered counterfactuals: 13612
Percentage filtered: 34.68%

=== Income 0 to 1 (<=50K to >50K) ===
Original counterfactuals: 16536
Filtered counterfactuals: 11119
Percentage filtered: 32.76%

Reasons for filtering:
Race changed: 664 (4.02%)
Gender changed: 1040 (6.29%)
Age decreased: 318 (1.92%)
Education decreased: 32 (0.19%)
Education-age mismatch: 3882 (23.48%)
Valid CFs: 11119

Race change details:
  Other->White: 225
  White->Other: 439

Gender change details:
  Male->Female: 482
  Female->Male: 558

=== Income 1 to 0 (>50K to <=50K) ===
Original counterfactuals: 4304
Filtered counterfactuals: 2493
Percentage filtered: 42.08%

Reasons for filtering:
Race changed: 328 (7.62%)
Gender changed: 258 (5.99%)
Age decreased: 850 (19.75%)
Education decreased: 449 (10.43%)
Education-age mismatch: 117 (2.72%)
Valid CFs: 2493

Race change details:
  White->Other: 315
  Other->White: 13

Gender change details:
  Female->Male: 55
  Male->Female: 203


In [35]:
continuous_features = ['age', 'hours_per_week']
all_predictors = x_train.columns.tolist()
categorical_features = list(set(all_predictors) - set(continuous_features))

nonlinear_validity_list = []
nonlinear_cont_prox_list = []
nonlinear_cat_prox_list = []
nonlinear_cont_div_list = []
nonlinear_cat_div_list = []
nonlinear_cont_count_div_list = []
nonlinear_sparsity_list = []

num_with_valid_cfs = 0
num_total = len(filtered_e_nonlinear.cf_examples_list)

for i, cf_exp in enumerate(filtered_e_nonlinear.cf_examples_list):
    query_instance = x_test.iloc[[i]]
    cf_df = cf_exp.final_cfs_df
    metrics = evaluate_cf_metrics(query_instance, cf_df, nonlinear, continuous_features, categorical_features, x_train, cont_count_threshold=1e-3)


    if metrics is not None:
        num_with_valid_cfs += 1
        nonlinear_validity_list.append(metrics['validity'])
        nonlinear_cont_prox_list.append(metrics['cont_proximity'])
        nonlinear_cat_prox_list.append(metrics['cat_proximity'])
        nonlinear_cont_div_list.append(metrics['cont_diversity'])
        nonlinear_cat_div_list.append(metrics['cat_diversity'])
        nonlinear_cont_count_div_list.append(metrics['cont_count_diversity'])
        nonlinear_sparsity_list.append(metrics['sparsity'])

nonlinear_avg_validity = np.mean(nonlinear_validity_list)
nonlinear_avg_cont_prox = np.mean(nonlinear_cont_prox_list)
nonlinear_avg_cat_prox = np.mean(nonlinear_cat_prox_list)
nonlinear_avg_cont_div = np.mean(nonlinear_cont_div_list)
nonlinear_avg_cat_div = np.mean(nonlinear_cat_div_list)
nonlinear_avg_cont_count_div = np.mean(nonlinear_cont_count_div_list)
nonlinear_avg_sparsity = np.mean(nonlinear_sparsity_list)

print(f"Instances with valid CFs after filtering: {num_with_valid_cfs}/{num_total} ({100*num_with_valid_cfs/num_total:.2f}%)")
print(f"Average Validity: {nonlinear_avg_validity*100:.2f}%")
print(f"Average Continuous Proximity: {nonlinear_avg_cont_prox:.4f}")
print(f"Average Categorical Proximity: {nonlinear_avg_cat_prox:.4f}")
print(f"Average Continuous Diversity: {nonlinear_avg_cont_div:.4f}")
print(f"Average Categorical Diversity: {nonlinear_avg_cat_div:.4f}")
print(f"Average Continuous Count Diversity: {nonlinear_avg_cont_count_div:.4f}")
print(f"Average Sparsity: {nonlinear_avg_sparsity:.4f}")

Instances with valid CFs after filtering: 4977/5210 (95.53%)
Average Validity: 100.00%
Average Continuous Proximity: -2.9397
Average Categorical Proximity: 0.8209
Average Continuous Diversity: 2.5932
Average Categorical Diversity: 0.1907
Average Continuous Count Diversity: 0.5390
Average Sparsity: 0.2496


In [42]:
with open("feasibility_nonlinear.pkl", "wb") as f:
    pickle.dump(filtered_e_nonlinear, f)

#Hybrid with SHAP

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import shap
import dice_ml
from dice_ml.utils import helpers
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [14]:
def compute_safe_shap_values(model, instance, x_train, num_background=1000):
    instance_transformed = model.named_steps['preprocessor'].transform(instance)

    background_data = x_train.sample(num_background, random_state=42)
    background_transformed = model.named_steps['preprocessor'].transform(background_data)

    def classifier_predict(X):
        return model.named_steps['classifier'].predict_proba(X)

    print("Creating SHAP explainer...")
    k_explainer = shap.KernelExplainer(classifier_predict, background_transformed)

    print("Calculating SHAP values...")
    shap_values = k_explainer.shap_values(instance_transformed)

    shap_shape = np.array(shap_values).shape
    print(f"SHAP values shape: {shap_shape}")

    if isinstance(shap_values, list):
        target_class = 1
        shap_values_for_features = shap_values[target_class][0]
    elif len(shap_values.shape) == 3:
        target_class = 1
        shap_values_for_features = shap_values[0, :, target_class]
    else:
        shap_values_for_features = shap_values[0]
    print("SHAP values successfully computed")
    return {
        'shap_values': shap_values,
        'shap_values_for_features': shap_values_for_features,
        'success': True
    }

In [15]:
def map_feature_importance(shap_result, instance, numerical_features, model):
    print("Mapping feature importance to original features...")

    importance_values = shap_result['shap_values_for_features']
    preprocessor = model.named_steps['preprocessor']

    cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
    categorical_features_train = cat_encoder.feature_names_in_

    current_categorical = [f for f in instance.columns if f not in numerical_features]

    categorical_features_ordered = [f for f in categorical_features_train if f in current_categorical]

    cat_feature_names = cat_encoder.get_feature_names_out(categorical_features_ordered)

    num_transformer = preprocessor.named_transformers_['num']
    num_features = num_transformer.get_feature_names_out(numerical_features)
    importance_mapping = {}

    for i, feat in enumerate(num_features):
        importance_mapping[numerical_features[i]] = importance_values[i]

    start_idx = len(num_features)
    for i, encoded_feat in enumerate(cat_feature_names):
        parts = encoded_feat.split('_')
        original_feat = '_'.join(parts[:-1])
        importance_mapping[original_feat] = importance_mapping.get(original_feat, 0) + importance_values[start_idx + i]

    importance_df = pd.DataFrame({
        'Feature': list(importance_mapping.keys()),
        'Importance': list(importance_mapping.values())
    }).sort_values(by='Importance', key=abs, ascending=False)

    print(f"Mapped {len(importance_mapping)} feature importance.")
    return importance_df


In [16]:
def analyze_counterfactual_alignment(instance, counterfactuals, importance_df, top_k=3):
    if importance_df.empty:
        print("Cannot analyze alignment - empty importance dataframe")
        return {
            'alignment_score': 0,
            'changes': [],
            'top_important_features': [],
            'success': False
        }

    if hasattr(counterfactuals, 'final_cfs_df'):
        cf_df = counterfactuals.final_cfs_df
    else:
        print("Cannot analyze alignment - couldn't extract counterfactual dataframe")
        return {
            'alignment_score': 0,
            'changes': [],
            'top_important_features': [],
            'success': False
        }


    top_important_features = importance_df.head(top_k)['Feature'].tolist()
    print(f"Top {top_k} important features: {top_important_features}")

    changes = []
    for i in range(len(cf_df)):
        cf = cf_df.iloc[i]
        changed_features = []

        for feature in cf.index:
            if feature not in ['income'] and cf[feature] != instance.iloc[0][feature]:
                changed_features.append(feature)

        important_features_changed = [f for f in changed_features if f in top_important_features]
        alignment_score = len(important_features_changed) / len(top_important_features) if top_important_features else 0

        changes.append({
            'CF': f'CF {i+1}',
            'Changed Features': changed_features,
            'Top Important Features Changed': important_features_changed,
            'Alignment Score': alignment_score
        })

    all_changed_features = set()
    for change in changes:
        all_changed_features.update(change['Changed Features'])

    overall_alignment = len([f for f in all_changed_features if f in top_important_features]) / len(top_important_features) if top_important_features else 0

    print(f"Overall alignment score: {overall_alignment:.2f}")
    return {
        'alignment_score': overall_alignment,
        'changes': changes,
        'top_important_features': top_important_features,
        'success': True
    }

In [17]:
def create_hybrid_table(instance, counterfactuals, importance_df):
    print("Creating hybrid table...")

    if importance_df.empty:
        print("Cannot analyze alignment - empty importance dataframe")
        return {
            'alignment_score': 0,
            'changes': [],
            'top_important_features': [],
            'success': False
        }

    cf_df = counterfactuals.final_cfs_df

    result_df = pd.DataFrame(instance.iloc[0])
    result_df.columns = ['Original']

    for i in range(len(cf_df)):
        cf_col = cf_df.iloc[i]
        result_df[f'CF {i+1}'] = cf_col

    importance_dict = dict(zip(importance_df['Feature'], importance_df['Importance']))
    result_df['Importance'] = result_df.index.map(lambda x: importance_dict.get(x, 0))

    result_df = result_df.sort_values('Importance', key=abs, ascending=False)

    print("Hybrid table created successfully")
    return result_df

In [31]:
def analyze_multiple_instances(instances, counterfactuals_list, model, x_train, numerical_features, model_name="Model"):
    print(f"\nAnalyzing {len(instances)} instances for {model_name}...")

    results = {
        'instances': [],
        'alignment_scores': [],
        'importance_features': [],
        'changed_features': [],
        'visualizations': [],
        'tables': []
    }

    for i, (instance, counterfactuals) in enumerate(zip(instances, counterfactuals_list)):
        print(f"\nProcessing instance {i+1}/{len(instances)}...")

        prob = model.predict_proba(instance)[0][1]
        print(f"prediction prob: {prob:.4f}")

        shap_result= compute_safe_shap_values(model, instance, x_train)

        importance_df = map_feature_importance(shap_result, instance, numerical_features, model)
        print(importance_df.head())

        alignment = analyze_counterfactual_alignment(instance, counterfactuals, importance_df)

        visualization = create_unified_hybrid_visualization(instance, counterfactuals, importance_df, model_name)

        hybrid_table = create_hybrid_table(instance, counterfactuals, importance_df)

        results['instances'].append(instance)
        results['alignment_scores'].append(alignment['alignment_score'])

        if not importance_df.empty:
            results['importance_features'].extend(importance_df.head(3)['Feature'].tolist())

        all_changes = []
        for change in alignment['changes']:
            all_changes.extend(change['Changed Features'])
        results['changed_features'].extend(all_changes)

        results['visualizations'].append(visualization)
        results['tables'].append(hybrid_table)

    results['avg_alignment'] = sum(results['alignment_scores']) / len(results['alignment_scores']) if results['alignment_scores'] else 0

    results['top_importance_features'] = Counter(results['importance_features']).most_common(10)

    results['top_changed_features'] = Counter(results['changed_features']).most_common(10)

    print(f"\nAnalysis complete for {model_name}")
    print(f"Average alignment score: {results['avg_alignment']:.4f}")
    print("Top important features:")
    for feature, count in results['top_importance_features'][:5]:
        print(f"  - {feature}: {count}")
    print("Top changed features:")
    for feature, count in results['top_changed_features'][:5]:
        print(f"  - {feature}: {count}")

    return results


In [32]:
def compare_models(model_results, models_to_compare):
    comparison = {
        'alignment_comparison': {},
        'importance_comparison': {},
        'changes_comparison': {},
        'overlap_metrics': {}
    }

    for model in models_to_compare:
        if model in model_results:
            comparison['alignment_comparison'][model] = model_results[model]['avg_alignment']

    importance_features = {}
    for model in models_to_compare:
        if model in model_results:
            importance_features[model] = set([f for f, _ in model_results[model]['top_importance_features']])

    changed_features = {}
    for model in models_to_compare:
        if model in model_results:
            changed_features[model] = set([f for f, _ in model_results[model]['top_changed_features']])

    if len(models_to_compare) > 1:
        for i, model1 in enumerate(models_to_compare):
            for model2 in models_to_compare[i+1:]:
                if model1 in importance_features and model2 in importance_features:
                    importance_overlap = len(importance_features[model1].intersection(importance_features[model2])) / len(importance_features[model1].union(importance_features[model2]))
                    comparison['overlap_metrics'][f'{model1}_vs_{model2}_importance'] = importance_overlap

                if model1 in changed_features and model2 in changed_features:
                    changes_overlap = len(changed_features[model1].intersection(changed_features[model2])) / len(changed_features[model1].union(changed_features[model2]))
                    comparison['overlap_metrics'][f'{model1}_vs_{model2}_changes'] = changes_overlap

    for model in models_to_compare:
        if model in model_results:
            comparison['importance_comparison'][model] = model_results[model]['top_importance_features']
            comparison['changes_comparison'][model] = model_results[model]['top_changed_features']

    print("Model comparison complete")
    for model in models_to_compare:
        if model in comparison['alignment_comparison']:
            print(f"{model} alignment score: {comparison['alignment_comparison'][model]:.4f}")

    if len(models_to_compare) > 1:
        print("\nFeature overlap between models:")
        for metric, value in comparison['overlap_metrics'].items():
            print(f"  - {metric}: {value:.4f}")

    return comparison

In [33]:
def create_comparison_visualizations(model_results, models_to_compare):
    print("\nCreating comparison visualizations...")

    visualizations = {}

    if len(models_to_compare) <= 1:
        print("Need at least two models to create comparison visualizations")
        return visualizations

    alignment_data = []
    for model in models_to_compare:
        if model in model_results:
            for score in model_results[model]['alignment_scores']:
                alignment_data.append({'Model': model, 'Alignment Score': score})

    if alignment_data:
        alignment_df = pd.DataFrame(alignment_data)
        print(alignment_df)

        fig1, ax1 = plt.subplots(figsize=(10, 6))
        sns.boxplot(x='Model', y='Alignment Score', data=alignment_df, ax=ax1)
        ax1.set_title('Alignment Score Comparison', fontsize=16)
        ax1.set_ylim(0, 1)
        visualizations['alignment_comparison'] = fig1

    importance_data = []
    for model in models_to_compare:
        if model in model_results:
            for feature, count in model_results[model]['top_importance_features']:
                importance_data.append({'Model': model, 'Feature': feature, 'Count': count})

    if importance_data:
        importance_df = pd.DataFrame(importance_data)

        pivot_importance = importance_df.pivot_table(
            index='Feature',
            columns='Model',
            values='Count',
            aggfunc='sum',
            fill_value=0
        )
        pivot_importance['Total'] = pivot_importance.sum(axis=1)
        pivot_importance = pivot_importance.sort_values('Total', ascending=False).drop('Total', axis=1)

        fig2, ax2 = plt.subplots(figsize=(12, 10))
        sns.heatmap(pivot_importance.head(15), annot=True, cmap='Blues', ax=ax2)
        ax2.set_title('Top Important Features Comparison', fontsize=16)
        visualizations['importance_comparison'] = fig2

    changes_data = []
    for model in models_to_compare:
        if model in model_results:
            for feature, count in model_results[model]['top_changed_features']:
                changes_data.append({'Model': model, 'Feature': feature, 'Count': count})

    if changes_data:
        changes_df = pd.DataFrame(changes_data)

        pivot_changes = changes_df.pivot_table(
            index='Feature',
            columns='Model',
            values='Count',
            aggfunc='sum',
            fill_value=0
        )
        pivot_changes['Total'] = pivot_changes.sum(axis=1)
        pivot_changes = pivot_changes.sort_values('Total', ascending=False).drop('Total', axis=1)

        fig3, ax3 = plt.subplots(figsize=(12, 10))
        sns.heatmap(pivot_changes.head(15), annot=True, cmap='Greens', ax=ax3)
        ax3.set_title('Top Changed Features Comparison', fontsize=16)
        visualizations['changes_comparison'] = fig3

    print(f"Created {len(visualizations)} comparison visualizations")
    return visualizations

In [34]:
def run_hybrid_explanation_analysis(models, model_names, instances, counterfactuals_dict, x_train, numerical_features=None):

    print("Starting hybrid explanation analysis...")

    if numerical_features is None:
        numerical_features = ["age", "hours_per_week"]

    results = {
        'model_results': {},
        'comparison': None,
        'comparison_visualizations': None
    }

    for model, model_name in zip(models, model_names):
        if model_name in counterfactuals_dict:
            counterfactuals_list = counterfactuals_dict[model_name]
            results['model_results'][model_name] = analyze_multiple_instances(
                instances, counterfactuals_list, model, x_train, numerical_features, model_name)

    if len(model_names) > 1:
        results['comparison'] = compare_models(results['model_results'], model_names)
        results['comparison_visualizations'] = create_comparison_visualizations(
            results['model_results'], model_names)

    print("\nHybrid explanation analysis complete!")
    return results

In [35]:
selected_indices = np.random.choice(np.where(logreg.predict(x_test) == 0)[0], size=100, replace=False)
instances = [x_test.iloc[[idx]] for idx in selected_indices]

In [36]:
import pickle

# with open("e_linear.pkl", "rb") as f:
#     e_logreg = pickle.load(f)

# with open("e_nonlinear.pkl", "rb") as f:
#     e_nonlinear = pickle.load(f)

logreg_counterfactuals = [e_logreg.cf_examples_list[idx] for idx in selected_indices]
nonlinear_counterfactuals = [e_nonlinear.cf_examples_list[idx] for idx in selected_indices]


In [40]:
counterfactuals_dict = {
    'Linear': logreg_counterfactuals,
    'Non-linear': nonlinear_counterfactuals
}

In [38]:
def create_unified_hybrid_visualization(instance, counterfactuals, importance_df, model_name="Model"):
    from matplotlib.patches import Patch
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    from matplotlib.colors import Normalize, LinearSegmentedColormap

    if importance_df.empty or not hasattr(counterfactuals, 'final_cfs_df'):
        return None
    cf_df = counterfactuals.final_cfs_df
    if cf_df.empty:
        return None


    features = instance.columns.tolist()
    importance_df = importance_df.set_index('Feature').loc[features].reset_index()

    num_cfs = len(cf_df)
    cf_labels = [f'CF {i+1}' for i in range(num_cfs)]

    colors = [(0.89, 0.10, 0.11), (1.0, 1.0, 1.0), (0.21, 0.46, 0.75)]
    cmap = LinearSegmentedColormap.from_list("custom_red_white_blue", colors, N=256)
    max_shap = max(abs(importance_df['Importance'].max()), abs(importance_df['Importance'].min()))
    norm = Normalize(vmin=-max_shap, vmax=max_shap)

    col_labels = [''] + features
    table_data = [
        ['Feature'] + features,
        ['Original'] + [instance[feature].values[0] for feature in features]
    ]

    cf_rows = []
    for i in range(num_cfs):
        cf = cf_df.iloc[i]
        cf_row = [f'CF {i+1}'] + [cf[feature] for feature in features]
        cf_rows.append(cf_row)

    cell_colors = []

    cell_colors.append(['#f0f0f0'] * len(col_labels))

    original_colors = ['white'] + [cmap(norm(importance_df.loc[importance_df['Feature'] == feature, 'Importance'].values[0])) for feature in features]
    cell_colors.append(original_colors)

    for cf_row in cf_rows:
        colors = ['white']
        for feature, cf_value in zip(features, cf_row[1:]):
            original_value = instance[feature].values[0]
            colors.append('lightgreen' if cf_value != original_value else 'white')
        cell_colors.append(colors)

    fig, ax = plt.subplots(figsize=(18, num_cfs * 1.2))
    ax.axis('off')

    all_rows = table_data + cf_rows

    table = ax.table(
        cellText=all_rows,
        cellColours=cell_colors,
        loc='center',
        cellLoc='center',
        colWidths=[0.08] + [0.92/len(features)]*len(features)
    )

    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.02, aspect=40)
    cbar.set_label('SHAP Value (Red: Negative, Blue: Positive)', fontsize=10)

    ax.legend(
        handles=[
            Patch(facecolor='lightgreen', label='Changed Feature'),
            Patch(facecolor='#f0f0f0', label='Feature Name')
        ],
        loc='upper right'
    )
    plt.title(f"{model_name} - Hybrid Explanation", fontsize=14, pad=20)
    plt.tight_layout(pad=2.0)
    return fig

In [ ]:
results = run_hybrid_explanation_analysis(
    models=[logreg, nonlinear],
    model_names=['Linear', 'Non-linear'],
    instances=instances,
    counterfactuals_dict=counterfactuals_dict,
    x_train=x_train,
    numerical_features=["age", "hours_per_week"]
)

In [ ]:
print(results)